In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

from sklearn.model_selection import train_test_split
from ast import literal_eval

import matplotlib.pyplot as plt

In [4]:
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

In [5]:
transactions_train['t_dat_datetime']  = pd.to_datetime(transactions_train['t_dat'])

transactions_train['t_month'] = transactions_train['t_dat_datetime'].dt.month

transactions_train['t_year']  = transactions_train['t_dat_datetime'].dt.year

In [6]:
transactions_train['customer_max_tran']  = transactions_train.\
groupby(['customer_id'])['t_dat_datetime'].transform('max')

In [7]:
cust_data = transactions_train[(transactions_train['customer_max_tran'].dt.year==2020)&
                                     (transactions_train['customer_max_tran'].dt.month == 9)]

cust_data['article_id'] = 'a'+cust_data['article_id'].astype('str')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [8]:
cust_data_2020_9 = cust_data.groupby(['customer_id','article_id']).agg(tran_count = ('t_dat_datetime','nunique')).\
assign(Rank = lambda x : x.groupby('customer_id')['tran_count'].rank(method='first', ascending=False)
      ).\
reset_index().\
query('Rank <= 12').\
groupby('customer_id').agg(prods = ('article_id','unique')).reset_index()

In [9]:
article_df =  pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
customer_df = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')

article_df['article_id'] = 'a'+article_df['article_id'].astype('str')

In [ ]:
cust_data = pd.merge(pd.merge(cust_data ,
         article_df[['article_id','product_group_name']],
         left_on = 'article_id',
         right_on = 'article_id'
        ),
         customer_df[['customer_id','age']],
         left_on = 'customer_id',
         right_on = 'customer_id'
        )

In [ ]:
cust_features = cust_data.groupby(['customer_id','age']).agg( tran_count = ('t_dat','nunique'),
                                       article_count = ('article_id','nunique'),
                                       spend = ('price','sum')
                                      ).reset_index()

In [ ]:
cust_features_1 = pd.pivot_table(cust_data[['customer_id','product_group_name','t_dat']],
         index = 'customer_id',
         columns = 'product_group_name',
         values = 't_dat',
         aggfunc = 'nunique'
        ).fillna(0).reset_index()

In [ ]:
cust_features = pd.merge(cust_features, 
                         cust_features_1,
                         left_on = 'customer_id',
                         right_on = 'customer_id',
                        )

In [ ]:
cust_features.head()

In [ ]:
cust_data_2020_9.iloc[:2]['prods'].apply(lambda x : list(x)).values

In [ ]:
cust_data_2020_9["prods"] = cust_data_2020_9["prods"].apply( lambda x : list(x) ) 
cust_data_2020_9["prods"].values[:5]

In [ ]:
model_data = pd.merge(cust_data_2020_9,
                      cust_features,
                      left_on = 'customer_id',
                      right_on = 'customer_id'
                     )

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [20]:
genre_mlb = mlb.fit_transform(model_data['prods'])
genre_mlb

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [1]:
genre = mlb.inverse_transform(genre_mlb[0])
genre

NameError: name 'mlb' is not defined

In [ ]:
top_12_products = transactions_train[(transactions_train['t_year']==2020)&
                                     (transactions_train['t_month'].isin([9]))
                                    ].groupby('article_id').agg({'customer_id':'nunique'}).reset_index().\
sort_values('customer_id', ascending=False).head(12)

top_12_products['article_id'] = " 0"+top_12_products.article_id.astype('str')

In [ ]:
submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

submission['prediction'] = top_12_products.article_id.sum()

submission.to_csv('submission.csv', index = False)